# Tarea 3 - MEL

Alejandro Mantilla - 201711304

Ximena Palacio - 201730995

## Índice

- [Parte A](#Parte-A.-Problemas-computacionales.)

    - [Problema 1](#Problema-1.-Bootstrap-para-pruebas-con-restricciones-lineales.)
    - [Problema 2](#Problem-2.-Método-de-estimación-robusta.)

- [Parte B](#Parte-B.-Problemas-teóricos,-conceptuales-y-experimentales.)

    - [Problema BONO](#Problema-BONO.-Uso-de-leverage-para-medidas-de-desempeño.)
    - [Problema 3](#Problema-3.-Problemas-de-especificación-y-selección-de-modelos.)
    
- [Parte C](#Parte-C.-Problemas-aplicados-con-datos-reales.)

    - [Problema 4](#Problema-4.-Ventas-de-secadoras-eléctricas-en-Bogotá.)
    - [Problema 5](#Problema-5.-Efectos-de-medidas-preventivas-en-la-severidad-del-coronavirus.)

## Parte A. Problemas computacionales.

### Problema 1. Bootstrap para pruebas con restricciones lineales.

### Problem 2. Método de estimación robusta.

## Parte B. Problemas teóricos, conceptuales y experimentales.

### Problema BONO. Uso de _leverage_ para medidas de desempeño.

### Problema 3. Problemas de especificación y selección de modelos.

## Parte C. Problemas aplicados con datos reales.

### Problema 4. Ventas de secadoras eléctricas en Bogotá.

### Problema 5. Efectos de medidas preventivas en la severidad del coronavirus.